In [1]:
import numpy as np
import pandas as pd

### 函数generate_time_series()生成的时间序列

In [2]:
'''生成一批时间序列数据，每个时间序列由两个正弦波的和以及一些噪声构成'''
def generate_time_series(batch_size,n_steps):
    freq1,freq2,offsets1,offsets2=np.random.rand(4,batch_size,1) #每个时间步只有一个值
    
    time=np.linspace(0, 1, n_steps) #生成时间步数组
    series=0.5*np.sin((time-offsets1)*(freq1*10+10)) #wave 1
    series+=0.2*np.sin((time-offsets2)*(freq2*20+20)) # +wave 2
    series+=0.1*(np.random.rand(batch_size,n_steps)-0.5) # +noise
    return series[...,np.newaxis].astype(np.float32) #返回 [批次大小，时间步数，1]，每个序列是2个正弦波之和（固定强度+随机频率和相位）+噪声
#在原始数组 series 的末尾添加了一个新的轴，从而改变了数组的维度
#series 是一个二维数组，形状为 (batch_size, n_steps)

#其形状是 批次大小 时间步数 维度 

### 创建训练集、验证集和测试集

In [3]:
n_steps=50
series=generate_time_series(10000,n_steps+1)
X_train,y_train=series[:7000,:n_steps],series[:7000,-1] #7000个时间序列，形状是[7000,50,1]
X_valid,y_valid=series[7000:9000,:n_steps],series[7000:9000,-1]
X_test,y_test=series[9000:,:n_steps],series[9000:,-1]

### 线性模型
使预测值是时间序列中每个值的线性组合

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,SimpleRNN,Conv1D,TimeDistributed,GRU,MaxPool1D
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd

In [6]:

'''使用 MSE 损失、Adam 优化器编译模型，在训练集上训练 20 个周期，用验证集评估'''
model=Sequential([Flatten(input_shape=[50,1]),Dense(1)])

model.compile(loss='mse',optimizer='adam')
history=model.fit(X_train,y_train,epochs=20,validation_data=(X_valid,y_valid))

model.evaluate(X_valid,y_valid)

Epoch 1/20
219/219 [==============================] - 1s 1ms/step - loss: 0.1244 - val_loss: 0.0445
Epoch 2/20
219/219 [==============================] - 0s 961us/step - loss: 0.0333 - val_loss: 0.0241
Epoch 3/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0209 - val_loss: 0.0167
Epoch 4/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0158 - val_loss: 0.0131
Epoch 5/20
219/219 [==============================] - 0s 919us/step - loss: 0.0128 - val_loss: 0.0109
Epoch 6/20
219/219 [==============================] - 0s 893us/step - loss: 0.0109 - val_loss: 0.0093
Epoch 7/20
219/219 [==============================] - 0s 940us/step - loss: 0.0094 - val_loss: 0.0081
Epoch 8/20
219/219 [==============================] - 0s 921us/step - loss: 0.0081 - val_loss: 0.0071
Epoch 9/20
219/219 [==============================] - 0s 905us/step - loss: 0.0071 - val_loss: 0.0064
Epoch 10/20
219/219 [==============================] - 0s 842us/step - loss: 0.0063 - va

0.003688463941216469

### 简单的RNN

In [9]:
'''
只有 1 个层，1 个神经元
不用指定输入序列的长度，因为循环神经网络可以处理任意的时间步（这就是为什么将第一个输入维度设为None）
'''
model=Sequential([SimpleRNN(1,input_shape=[None,1])])

optimizer=Adam(learning_rate=0.005)
model.compile(loss='mse',optimizer=optimizer)
history=model.fit(X_train,y_train,epochs=20,validation_data=(X_valid,y_valid))

model.evaluate(X_valid,y_valid)

Epoch 1/20
219/219 [==============================] - 2s 7ms/step - loss: 0.3680 - val_loss: 0.2544
Epoch 2/20
219/219 [==============================] - 1s 7ms/step - loss: 0.1791 - val_loss: 0.1203
Epoch 3/20
219/219 [==============================] - 2s 8ms/step - loss: 0.0804 - val_loss: 0.0490
Epoch 4/20
219/219 [==============================] - 2s 7ms/step - loss: 0.0312 - val_loss: 0.0195
Epoch 5/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0148 - val_loss: 0.0126
Epoch 6/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0118 - val_loss: 0.0118
Epoch 7/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0115 - val_loss: 0.0117
Epoch 8/20
219/219 [==============================] - 2s 11ms/step - loss: 0.0114 - val_loss: 0.0117
Epoch 9/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0114 - val_loss: 0.0117
Epoch 10/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0114 - val_loss: 

0.011659803800284863

### 深度RNN
3个SimpleRNN

In [10]:
'''
所有循环层一定要设置return_sequences=True（除了最后一层，因为最后一层只关心输出
如果没有设置，输出的是 2D 数组（只有最终时间步的输出），而不是3D数组（包含所有时间步的输出），下一个循环层就接收不到 3D 格式的序列数据。
'''
model=Sequential([
    SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
    SimpleRNN(20,return_sequences=True),
    SimpleRNN(1)
])

model.compile(loss='mse',optimizer='adam')
history=model.fit(X_train,y_train,epochs=20,validation_data=(X_valid,y_valid))

model.evaluate(X_valid,y_valid)

Epoch 1/20
219/219 [==============================] - 6s 21ms/step - loss: 0.0271 - val_loss: 0.0083
Epoch 2/20
219/219 [==============================] - 7s 30ms/step - loss: 0.0070 - val_loss: 0.0053
Epoch 3/20
219/219 [==============================] - 9s 41ms/step - loss: 0.0053 - val_loss: 0.0043
Epoch 4/20
219/219 [==============================] - 8s 35ms/step - loss: 0.0044 - val_loss: 0.0039
Epoch 5/20
219/219 [==============================] - 4s 19ms/step - loss: 0.0042 - val_loss: 0.0037
Epoch 6/20
219/219 [==============================] - 4s 18ms/step - loss: 0.0038 - val_loss: 0.0034
Epoch 7/20
219/219 [==============================] - 5s 21ms/step - loss: 0.0036 - val_loss: 0.0034
Epoch 8/20
219/219 [==============================] - 7s 30ms/step - loss: 0.0036 - val_loss: 0.0032
Epoch 9/20
219/219 [==============================] - 6s 28ms/step - loss: 0.0035 - val_loss: 0.0033
Epoch 10/20
219/219 [==============================] - 4s 18ms/step - loss: 0.0035 - val_lo

0.002830996410921216

In [11]:
'''
所有循环层一定要设置return_sequences=True（除了最后一层，因为最后一层只关心输出
如果没有设置，输出的是 2D 数组（只有最终时间步的输出），而不是3D数组（包含所有时间步的输出），下一个循环层就接收不到 3D 格式的序列数据。
'''
model=Sequential([
    SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
    SimpleRNN(20),
    Dense(1)
])

model.compile(loss='mse',optimizer='adam')
history=model.fit(X_train,y_train,epochs=20,validation_data=(X_valid,y_valid))

model.evaluate(X_valid,y_valid)

Epoch 1/20
219/219 [==============================] - 4s 13ms/step - loss: 0.0168 - val_loss: 0.0054
Epoch 2/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0046 - val_loss: 0.0046
Epoch 3/20
219/219 [==============================] - 6s 27ms/step - loss: 0.0038 - val_loss: 0.0040
Epoch 4/20
219/219 [==============================] - 7s 33ms/step - loss: 0.0036 - val_loss: 0.0031
Epoch 5/20
219/219 [==============================] - 5s 23ms/step - loss: 0.0034 - val_loss: 0.0037
Epoch 6/20
219/219 [==============================] - 4s 20ms/step - loss: 0.0033 - val_loss: 0.0030
Epoch 7/20
219/219 [==============================] - 5s 21ms/step - loss: 0.0032 - val_loss: 0.0032
Epoch 8/20
219/219 [==============================] - 4s 21ms/step - loss: 0.0032 - val_loss: 0.0028
Epoch 9/20
219/219 [==============================] - 5s 21ms/step - loss: 0.0031 - val_loss: 0.0029
Epoch 10/20
219/219 [==============================] - 4s 20ms/step - loss: 0.0030 - val_lo

0.0027142001781612635

### 使用1D卷积层处理序列

In [ ]:
n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

In [ ]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

In [3]:
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
                        input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train[:, 3::2], epochs=20,
                    validation_data=(X_valid, Y_valid[:, 3::2]))

TypeError: ('Keyword argument not understood:', 'input_size')

### 1维卷积神经网络，并对随机数据进行处理

In [ ]:
#卷积层
filters=1
kernel_size=5
convolution_1d_layer=Conv1D(filters,kernel_size,strides=1,input_shape=[25,1],padding='VALID',activation='relu')
#池化层
max_pooling_layer=MaxPool1D(pool_size=5,strides=1,padding='VALID')

#创建一维向量，并扩展维度为(1,25,1)
data_1d=np.random.normal(size=25)
data_1d=np.expand_dims(data_1d,0)
data_1d=np.expand_dims(data_1d,2)

output=model.predict(data_1d)
print(output)

In [ ]:
import numpy as np
#创建一维向量，并扩展维度为(1,25,1)
data_1d=np.random.normal(size=25)
data_1d

array([-1.77548559, -0.57999813, -0.51110387,  0.21865217, -0.14772994,
        2.56035814, -1.63324737,  1.06723355, -1.12770584,  0.03411521,
       -0.17187189,  0.9176972 ,  0.8620269 ,  0.76294606,  2.83128853,
       -0.6556783 ,  1.74314179, -0.44547304, -0.84686993, -0.23310408,
        1.66333698,  1.76469115,  0.90672214,  1.22336184,  1.0249977 ])

In [ ]:
data_1d=np.expand_dims(data_1d,0)
data_1d

array([[-1.77548559, -0.57999813, -0.51110387,  0.21865217, -0.14772994,
         2.56035814, -1.63324737,  1.06723355, -1.12770584,  0.03411521,
        -0.17187189,  0.9176972 ,  0.8620269 ,  0.76294606,  2.83128853,
        -0.6556783 ,  1.74314179, -0.44547304, -0.84686993, -0.23310408,
         1.66333698,  1.76469115,  0.90672214,  1.22336184,  1.0249977 ]])

In [ ]:
data_1d=np.expand_dims(data_1d,2)
data_1d

array([[[-1.77548559],
        [-0.57999813],
        [-0.51110387],
        [ 0.21865217],
        [-0.14772994],
        [ 2.56035814],
        [-1.63324737],
        [ 1.06723355],
        [-1.12770584],
        [ 0.03411521],
        [-0.17187189],
        [ 0.9176972 ],
        [ 0.8620269 ],
        [ 0.76294606],
        [ 2.83128853],
        [-0.6556783 ],
        [ 1.74314179],
        [-0.44547304],
        [-0.84686993],
        [-0.23310408],
        [ 1.66333698],
        [ 1.76469115],
        [ 0.90672214],
        [ 1.22336184],
        [ 1.0249977 ]]])

### WaveNet

In [ ]:
model=Sequential()
for rate in (1,2,4,8)*2:
    model.add(Conv1D(filters=20,
                     kernel_size=2,
                     padding='causal',
                     activation='relu',
                     dilation_rate=rate,
                     input_shape=[None,1]))

model.add(Conv1D(filters=10,kernel_size=1))

model.compile(loss='mse',optimizer='adam',metrics=[last_time_step_mse])
history=model.fit(X_train,y_train,epochs=20,
                  validation_data=(X_valid,y_valid))